# Random Forest Analysis

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn.model_selection import train_test_split
import ast
import statsmodels.api as sm
from scipy import stats
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as pyplot
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Choose the Organization of Interest

In [ ]:
org = "USC"

# Load Account Data

In [ ]:
# Load Account Data For a Specific Organization

account = pd.read_csv("../sample_data/other_samples/acc_tags_set.csv")
account = account[account.org_id==org].copy()
#account.drop(columns = ["nan_tag"], inplace = True)

interested_tags = set()
acc_tag_dict = defaultdict(int)
for i in range(len(account)):
    tag_list = ast.literal_eval(account.tags.iloc[i])
    for t in tag_list:
        acc_tag_dict[t]+=1
        interested_tags.add(t)
account_per_tag = []
for k,v in acc_tag_dict.items():
    account_per_tag.append({"tag":k,"counts":v })
account_per_tag = pd.DataFrame(account_per_tag)

# Load Seasonal Ticket Data

In [ ]:
ticket_season = pd.read_csv(f"../sample_data/other_samples/{org}_ticket_trans_item_event.csv")

In [ ]:
ticket_season = ticket_season[ticket_season.org_id.isin([org])].copy()
ticket_orgs = ticket_season.org_id.unique()

# Preprocess Data

## Only select the tags that have been assigned to more than 100 accounts

In [ ]:
useful_tags = []
error_tags = []
# method = "logit"

for i,tag in enumerate(interested_tags):
    if type(tag)==str and account_per_tag[account_per_tag.tag==tag].iloc[0]["counts"] >100:
        useful_tags.append(tag)

## Pivot Wide Table so that each column is a season of the organization

In [ ]:
ticket_season_org = ticket_season[["account_id","org_id","season","total_epay",
                                   "e_oqty"]].groupby(["account_id","org_id",
                                                       "season"]).sum().reset_index()
ticket_season_org = ticket_season_org.pivot_table(index = ["account_id","org_id"], 
                           columns = ["season"],
                           values = ["total_epay","e_oqty"]).fillna(0)
ticket_season_org.columns = ["_".join(x) if x[0] not in ["account_id","org_id"] else x[0] for x in ticket_season_org.columns.ravel()]
temp = pd.merge(account,ticket_season_org, on = ["account_id","org_id"], how = "inner")
temp.fillna(0,inplace = True)

# Random Forest

In [ ]:
def get_roc(testy, pred_y, org , tag):
    ns_probs = [0 for _ in range(len(testy))]
    # calculate scores
    ns_auc = roc_auc_score(testy, ns_probs)
    lr_auc = roc_auc_score(testy, pred_y)
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print('Logistic: ROC AUC=%.3f' % (lr_auc))
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(testy, pred_y)
    return lr_auc,ns_fpr, ns_tpr,lr_fpr, lr_tpr

def plot_roc(ns_fpr, ns_tpr,lr_fpr, lr_tpr, tag, org):
    
    colors = ["#2CBDFE","#47DBCD","#F3A0F2","#9D2EC5","#F5B14C"]
    # plot the roc curve for the model
    for i,(lf, lt,color) in enumerate(zip(lr_fpr, lr_tpr, colors)):      
        pyplot.plot(lf, lt, marker='.', label=f'CV {i+1}', color=color )
    pyplot.plot(ns_fpr[0], ns_tpr[0], linestyle='dashdot', label='Logistic',color = "black")
    # axis labels
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    # show the legend
    pyplot.legend()
    
    pyplot.title(f"ROC Curve for Tag {tag} in {org}")
    # save image
    pyplot.savefig(f'../outputs/Random Forest/{org}_{tag}_ROC.png',bbox_inches='tight')
    
    # show the plot
    
    pyplot.show()
#     return lr_auc

In [ ]:
def calculate_season_rf(tag,org):
    '''Get 5-fold Cross Validation Result for Random Forest in predicting whether the accounts
    in a selected organization are assigned a selected tag.
    Get the account and ticket data from previous codes to avoid copying too many data and crashing the session
    '''
    print("Selected Tag:", tag)
#     ----------------------------------Preprocess Data ----------------------------------
#     Ticket Season: Number of Columns
    print("old columns number:",len(ticket_season_org.columns))
    
#     Account Data: Create Target Column to indicate whether and again return 0 if 
    tag_account = account.copy()
    tag_account["tag"] = tag_account.tags.str.contains(tag).fillna(False)
    tag_num = len(tag_account[tag_account.tag])
    if tag_num<100:
        return 0
    
#     Merge Account + Ticket Season    
    temp = pd.merge(tag_account,ticket_season_org, on = ["account_id","org_id"], how = "left")
    total_number_of_account = len(temp)
    total_number_of_acc_w_tag = len(temp[temp.tag])
    print("total number of accounts labeled with tag:", total_number_of_acc_w_tag)
    print("total number of accounts: ",total_number_of_account)
    print("percentage of tags:",total_number_of_acc_w_tag/total_number_of_account)
    
    if len(temp[temp.tag])<100:     
        return 0
    
    temp.fillna(0,inplace = True)   
    
#     Drop Season Columns if fewer than 1000 accounts have bought relevant tickets:
    temp["intercept"] = 1
    temp.drop(columns = ["tags"], inplace = True)
    drop_cols = []
    for c in temp.columns:
        if c not in ["account_id","org_id",'tag']:
            if len(temp[temp[c]>0])<1000:
                drop_cols.append(c)
    print(drop_cols)
    temp.drop(columns = drop_cols, inplace = True)
    
#     The X variable Columns
    xs = [c for c in temp.columns if c not in ["account_id","org_id",'tag']]
    print("new columns number:",len(xs))

#     Randomize the dataframe with sample and fraction = 1 and replace = False
    np.random.seed(0)
    temp = temp.sample(frac=1).reset_index(drop=True)
    
#     random forest
    overall_result=RandomForestClassifier().fit(temp[xs],temp.tag)
    
#    -------------------------------------------------------------------------------------------    
#     ------------------5-FOLD Cross Validation Data Preprocess-------------------

#     Split the data with tag from no tag for future 5-fold splitting
    temp_tag = temp[temp.tag].copy()
    temp = temp[~temp.tag]

#     # Tags Train Split - index
    arr_tags = np.arange(len(temp_tag))
    np.random.seed(0)
    np.random.shuffle(arr_tags)
    arr_tags=np.array_split(arr_tags, 5)
    
#     # Not Tags Train Split - index
    arr_no_tags = np.arange(len(temp))
    np.random.seed(0)
    np.random.shuffle(arr_no_tags)
    arr_no_tags=np.array_split(arr_no_tags, 5)
    
    temp.reset_index(inplace = True)
    temp_tag.reset_index(inplace = True)

#     ------------------5-FOLD Cross Validation and Metrics-------------------
    
    average_mse =0
    avg_specificity = 0
    avg_sensitivity = 0
    avg_f1 = 0
    sense_gd=True
    spec_gd = True
    avg_auc_score=0
    ns_fprs, ns_tprs,lr_fprs, lr_tprs = [],[],[],[]
    for i,(ari1, ari2) in enumerate(zip(arr_no_tags, arr_tags)):
#         Train Test split using the previously prepared indexes in each fold
        temp_test = pd.concat([temp.iloc[ari1],
                             temp_tag.iloc[ari2]]).reset_index()
        temp_tr = pd.concat([temp.drop(ari1,axis =0),
                               temp_tag.drop(ari2,axis =0)])
        temp_tr = temp_tr.sample(frac=1).reset_index(drop=True)

#       Train random forest Model 
        result=RandomForestClassifier().fit(temp_tr[xs],temp_tr.tag)
        
        sorted_idx = result.feature_importances_.argsort()
        plt.barh([xs[c] for c in sorted_idx], result.feature_importances_[sorted_idx])
        plt.xlabel("Random Forest Feature Importance")
        plt.show()
        predicted = result.predict_proba(temp_test[xs])[:,1]
        
#        Calculate auc score and plot roc curve

        auc_score, ns_fpr,ns_tpr,lr_fpr,lr_tpr = get_roc(temp_test.tag,predicted,org,tag)
        ns_fprs.append(ns_fpr)
        ns_tprs.append(ns_tpr)
        lr_tprs.append(lr_tpr)
        lr_fprs.append(lr_fpr)
        avg_auc_score+=auc_score
        
#           Use 0.5 as threshold: <0.5 means no tag; >=0.5 means tag
        predicted = np.where(predicted<0.5,0,1)
        
#         Calculate Measure Metrics:
#       mse, true_positive, false_negative,true_negative,false_positive
        mse = np.sum((predicted - temp_test.tag)**2)/len( temp_test.tag)

        average_mse+=mse
        true_positive = 0
        false_negative = 0
        true_negative = 0
        false_positive= 0
        
        for i in range(len(predicted)):
            if predicted[i] and temp_test.tag.iloc[i]:
                true_positive+=1
            elif not predicted[i] and not temp_test.tag.iloc[i]:
                true_negative+=1
            elif predicted[i] and not temp_test.tag.iloc[i]:
                false_negative+=1
            else:
                false_positive+=1
        print(true_positive,false_negative,true_negative,false_positive)
        if true_positive+false_negative!=0:
            sensitivity = true_positive/(true_positive+false_negative)
            avg_sensitivity+=sensitivity
#             print(i,"sensitivity:",sensitivity)
        else:
            sense_gd=False
        if true_negative+false_positive!=0:
            specificity = true_negative/(true_negative+false_positive) 
            avg_specificity+=specificity
#             print(i,"specificity:",specificity)
        else:
            spec_gd = False
            
        avg_f1 += true_positive/(true_positive+(false_positive+false_negative)/2)

    plot_roc(ns_fprs,ns_tprs,lr_fprs,lr_tprs,tag,org)
    if sense_gd:
        avg_sensitivity = avg_sensitivity/5
        print("avg_sensitivity:",avg_sensitivity)
    if spec_gd:
        avg_specificity = avg_specificity/5
        print("avg_specificity:",avg_specificity)
        
    avg_f1 = avg_f1/5
    print("avg_f1 score:", avg_f1)
              
    average_mse = average_mse/5
    print("average_mse score:", average_mse)
    
    avg_auc_score = avg_auc_score/5
    print("avg_auc_score:",avg_auc_score)
    
    result = {"overall_result":overall_result, 
              #"prsquared": overall_result.prsquared, 
              "total_number_of_account":total_number_of_account,
              "total_number_of_acc_w_tag":total_number_of_acc_w_tag,
              "Percentage of Tag": total_number_of_acc_w_tag/total_number_of_account,
              "Correct Percentage":average_mse,
              "avg_f1":avg_f1,
              "avg_auc":avg_auc_score,
              "avg_sensitivity":avg_sensitivity,
              "avg_specificity":avg_specificity,
              "sense_gd":sense_gd, 
              "spec_gd":spec_gd}
    #print("feature importance plot:", feature_importance(result))
    return result, temp_test

In [ ]:
tags_dict = dict()
for tag in ['fbbb','fbonly']:
    print(tag)
    result, temp_test = calculate_season_rf(tag,org)
#     print(result)
    if result!=0:
        tags_dict[tag] = result

## Format Result (optional)

In [ ]:
def format_model_result(tags_dict, sort_by_cols=["avg_sensitivity","avg_specificity"]):
    temp = []
    for k,v in tags_dict.items():
        tags_dict[k]["tag"]=k
        temp.append(tags_dict[k])
    tags_dict = pd.DataFrame(temp)
    return tags_dict.sort_values(by = sort_by_cols,ascending = False)

## Show Result

In [ ]:
format_model_result(tags_dict, ["avg_sensitivity","avg_specificity"]).to_csv(
    f"../outputs/Random Forest/tag_prediction_result_{org}.csv")